In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

43809


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [18]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
# add_alternate_names = (
#     "INSERT INTO alternative_name "
#     "(name, taxonomy, gene, splice, histone) "
#     "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
# )
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

In [14]:
query = "SELECT * FROM publication "
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].str.startswith("hara")]

,id,title,doi,author,year,pubmed_id


In [70]:
query = "SELECT * FROM histone_description "
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
# df
for s in df.apply(lambda row: "".join(row.astype(str)), axis=1):
    if "henikoff_heterochromatic_2000" in s:
        print(s)
        print("---------------")

In [41]:
query = "SHOW TABLES; "
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,Tables_in_db_name
0,alternative_name
1,histone
2,histone_description
3,histone_has_publication
4,publication
5,sequence
6,sequence_has_publication


# To Do H3-like

## <span style="color:green">Update description to H3.Y.1_(Homo_sapiens)</span>

### <span style="color:green">Update knock_out</span>

```Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024].```

### <span style="color:green">Update function</span>

```Although the functions of H3.Y.1 remain poorly understood, it is known to promote cell growth and regulate the expression of genes involved in cell cycle control and mitosis [wiedemann_identification_2010]. The presence of H3.Y.1 (along with H3.Y.2) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. Furthermore, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also detected in vivo within the nuclei of human 8-cell embryos at the peak of ZGA [taubenschmid-stowers_8c-like_2022]. During blastomere division, H3.Y.1 and H3.Y.2 show strong association with condensed chromosomes at prophase and metaphase stages [taubenschmid-stowers_8c-like_2022]. Taubenschmid-Stowers et al. suggest that H3.Y.1 and H3.Y.2 may be necessary for large-scale genome activation during early human embryogenesis [taubenschmid-stowers_8c-like_2022].```

### <span style="color:green">Add disease</span>

```H3.Y.1 and H3.Y.2 are critical epigenetic amplifiers in facioscapulohumeral dystrophy (FSHD) pathogenesis [kong_engineered_2024]. They incorporate into chromatin to activate late targets like LEUTX, which in turn stimulates H3.Y.1 and H3.Y.2 expression, forming a pathogenic positive feedback loop. This coherent feedforward network with DUX4 and LEUTX potently stabilizes and enhances the aberrant FSHD gene expression signature [kong_engineered_2024.```

## <span style="color:green">Update description to H3.Y.2_(Homo_sapiens)</span>

### <span style="color:green">Update knock_out</span>

```Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024].```

### <span style="color:green">Update function</span>

```The functions of H3.Y.2 remain poorly understood. However, the presence of H3.Y.2 (along with H3.Y.1) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. In addition, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also detected in vivo within the nuclei of human 8-cell embryos at the peak of ZGA [taubenschmid-stowers_8c-like_2022]. During blastomere division, H3.Y.1 and H3.Y.2 show strong association with condensed chromosomes at prophase and metaphase stages [taubenschmid-stowers_8c-like_2022]. Taubenschmid-Stowers et al. suggest that H3.Y.1 and H3.Y.2 may be necessary for large-scale genome activation during early human embryogenesis [taubenschmid-stowers_8c-like_2022].```

### <span style="color:green">Add disease</span>

```H3.Y.1 and H3.Y.2 are critical epigenetic amplifiers in facioscapulohumeral dystrophy (FSHD) pathogenesis [kong_engineered_2024]. They incorporate into chromatin to activate late targets like LEUTX, which in turn stimulates H3.Y.1 and H3.Y.2 expression, forming a pathogenic positive feedback loop. This coherent feedforward network with DUX4 and LEUTX potently stabilizes and enhances the aberrant FSHD gene expression signature [kong_engineered_2024.```

## <span style="color:black">ВАЖНО!!!</span>

### Предыдущее описание H3.Y:

**Summary:** H3.Y is involved in memory formation due to its presence in neurons in human hippocampus. One splice isoform has an extended C-terminal alpha tail that may interacts with the H2A.Z acidic patch
**Taxonomic span:** Primates, 9443
**Alternate names:** H3.X

# To Do H3

## <span style="color:green">Add description to H3.4_(Homo_sapiens)</span>

### <span style="color:green">Add summary</span>

```H3.4_(Homo_sapiens) is a human H3 histone variant highly expressed in testis and at lower level in other tissues.```

### <span style="color:green">Add gene</span>

```H3.4 is encoded by H3-4 gene (formerly HIST3H3, also an alias name H3C16 has been suggested (Seal et al.)), located on chromosome 1 adjacent to genes H2AC25 and H2BC26 in a small histone gene cluster [kycia_tudor_2014]. This gene is intronless, its mRNA has a stem-loop structure.```

### <span style="color:green">Add deposition</span>

```Tachiwana et al. have found that H3.4/H4 is not efficiently incorporated into the nucleosome by human Nap1 chaperone, due to its defective H3.4/H4 deposition on DNA. In contrast, human Nap2 chaperone, a paralog of Nap1, promoted nucleosome assembly with H3.4/H4. Mutational analyses revealed that the A111V mutation in H3.4 is essential for this difference [tachiwana_nucleosome_2008].```

### <span style="color:green">Add structure</span>

```PDB structure of human H3.4 containing nucleosome is available with id 3A6N. Structural studies have revealed that H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].```

### <span style="color:green">Add sequence</span>

```In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V) [ueda_testis-specific_2017].```

### <span style="color:green">Add disease</span>

```The single‑nucleotide polymorphism c190C>T (Arg64Cys) in the human testis‑specific histone variant, H3.4, was studied in Japanese patients for its association with Sertoli cell‑only syndrome, which causes infertility [miyamoto_association_2018].```

## <span style="color:green">Update description to H3.4_(Mammalia)</span>

### <span style="color:green">Update summary</span>

```H3.4_(Mammalia) is a mammal-specific H3 histone variant highly expressed in testis and at lower level in other tissues [kycia_tudor_2014]. This variant is also reffered to as H3.1t, H3T, or TS H3.4. The H3.4 variant is very similar to canonical H3 histones both in sequence, gene location and transcriptional regulation. In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V), its gene is located within histone gene cluster, is intronless, its mRNA has a stem-loop structure. H3.4 expression is replication-dependent [kycia_tudor_2014, ueda_testis-specific_2017].```

### <span style="color:green">Update knock_out</span>

```Knockout mice for H3.4 were first generated in 2017; both male and female H3t null mice were viable and healthy, but the male mice were sterile. H3.4 deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024, ueda_testis-specific_2017].```

### <span style="color:green">Update function</span>

```H3.4 is mainly responsible for facilitating the transition from spermatogonia stem cells to active spermatogenesis. Mouse H3.4 was shown to be essential for very early stages of spermatogenesis, its deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024]. When differentiating spermatogonia emerge in normal spermatogenesis, H3.4 appears and replaces the canonical H3 proteins [ueda_testis-specific_2017].```

### <span style="color:green">Update expression</span>

```Expression of H3.4 is strictly restricted to testicular tissue. In mice, it is detected in spermatogonia, meiotic cells, and haploid round spermatids. However, its expression level decreases significantly in early elongating spermatids, indicating stage-specific regulation during spermatogenesis [kawaguchi_comprehensive_2024].```

### <span style="color:green">Update localization</span>

```In mouse H3.4 is not incorporated into the XY body, which forms during pachytene of meiotic prophase I, suggesting a potential role for H3.4 in organizing active chromatin regions [kawaguchi_comprehensive_2024].```

### <span style="color:green">Update interactions</span>

```It was demostrated that A23V mutation favors the interaction of trymethyllysine-binding Tudor domains of PHF1 and PHF2 proteins (critical components of the PCR2 complex) with H3K27me3 marks. PHF1 co-localizes with H3.4 in testis and its Tudor domain preferentially binds to H3.4K27me3 over canonical H3K27me3 in vitro, implicating that H3.4K27me3 might be a physiological ligand of PHF1/19. [dong_structural_2020, kycia_tudor_2014].```

### <span style="color:green">Add structure</span>

```Structural studies have revealed that human H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].```

### <span style="color:green">Update disease to NULL</span>

## <span style="color:green">Update description to H3.5_(Hominidae)</span>

### <span style="color:green">Update summary</span>

```H3.5 is a hominid-specific histone H3 variant, also known as H3.3C, expressed in the seminiferous tubules of the human testis and plays a role in regulating spermatogenesis. H3.5 likely evolved from H3F3B gene duplication and is associated with euchromatin and actively transcribed genes [schenk_h35_2011].```

### <span style="color:green">Update sequence</span>

```H3.3 is the most similar variant to H3.5 (approximately 96% identity). H3.5 has two copies of the ARKST motif and 5 amino acids differences comparing to H3.3 [schenk_h35_2011, ederveen_human_2011].```

## <span style="color:green">Add description to H3.5_(Homo_sapiens)</span>

### <span style="color:green">Add summary</span>

```H3.5_(Homo_sapiens) is a H3 histone variant expressed in the seminiferous tubules of human testes.```

### <span style="color:green">Add function</span>

```H3.5 plays a significant role in testicular cell differentiation, with potential implications in the pathogenesis of testicular diseases.```

### <span style="color:green">Add expression</span>

```The highest expression of H3.5 is found in spermatocytes, while lower levels are present in early and late spermatids, and absent in mature spermatozoa [weil_histone_2024, ding_primate-specific_2021]. However, low-abundance H3.5 transcripts were also identified in leukocytes and hepatocytes [schenk_h35_2011].```

### <span style="color:green">Add gene</span>

```H3.5 is encoded by H3-5 gene on chromosome 12. Methylation of the CpG island in the promoter region of the gene plays a key role in its tissue-specific expression regulation [weil_histone_2024].```

### <span style="color:green">Update sequence</span>

```H3.5 differs from H3.3 by five amino acid substitutions: Thr29, Cys33, Asn78, Val88, and Leu103 (which correspond to Ala29, Gly33, Lys79, Ile89, and Phe104 in H3.3, respectively) [urahama_histone_2016]. Furthermore, H3.5 lacks the lysine residue at position 37, which is present in the major H3 variants.```

### <span style="color:green">Update localization</span>

```H3.5 localizes primarely at exons, 5'UTR, 3'UTR, promoters and introns. Although it enreached around transcription start sites (TSSs), H3.5 distribution is independent of gene expression levels [urahama_histone_2016].```

### <span style="color:green">Add deposition</span>

```H3.5 physically interacts with histone chaperones Asf1a, Asf1b, HIRA, CAF-1 p150, and DAXX, suggesting its involvement in chromatin remodeling and gene regulation processes [weil_histone_2024]. An unpublished experiment by Ding et al. demonstrated that the chaperone NAP1L1 exhibits a higher binding affinity for the H3.5/H4 tetramer compared to the H3.3/H4 tetramer, attributable to amino acid alterations within the N-terminal region of H3.5 [ding_primate-specific_2021].```

### <span style="color:green">Update structure</span>

```Nucleosomes containing histone variant H3.5 are less stable than H3.1 and H3.3 nucleosomes, but more stable than H3.4 nucleosome [urahama_histone_2016]. Notable, amino acid L in position 103 of H3.5 (amino acid F at the corresponding position in H3.3) reduces the number of hydrophobic bonds with histone H4 leading to instability of nucleosome [urahama_histone_2016].```

### <span style="color:green">Update disease</span>

```Clinical data indicate that H3.5 mRNA levels are significantly lower in patients with nonobstructive azoospermia compared to those with obstructive azoospermia or normal individuals, suggesting H3.5's role in normal spermatogenesis and its regulation by gonadotropins [ding_primate-specific_2021]. Elevated H3.5 expression is observed in seminomas, and H3-5 gene copy number gain is associated with its increased expression in tumor tissues [weil_histone_2024].```

# Done

## <span style="color:black">Update description to H3.Y.1_(Homo_sapiens)</span>

### <span style="color:black">Update knock_out</span>

```Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024].```

### <span style="color:black">Update function</span>

```Although the functions of H3.Y.1 remain poorly understood, it is known to promote cell growth and regulate the expression of genes involved in cell cycle control and mitosis [wiedemann_identification_2010]. The presence of H3.Y.1 (along with H3.Y.2) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. Furthermore, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also detected in vivo within the nuclei of human 8-cell embryos at the peak of ZGA [taubenschmid-stowers_8c-like_2022]. During blastomere division, H3.Y.1 and H3.Y.2 show strong association with condensed chromosomes at prophase and metaphase stages [taubenschmid-stowers_8c-like_2022]. Taubenschmid-Stowers et al. suggest that H3.Y.1 and H3.Y.2 may be necessary for large-scale genome activation during early human embryogenesis [taubenschmid-stowers_8c-like_2022].```

### <span style="color:black">Add disease</span>

```H3.Y.1 and H3.Y.2 are critical epigenetic amplifiers in facioscapulohumeral dystrophy (FSHD) pathogenesis [kong_engineered_2024]. They incorporate into chromatin to activate late targets like LEUTX, which in turn stimulates H3.Y.1 and H3.Y.2 expression, forming a pathogenic positive feedback loop. This coherent feedforward network with DUX4 and LEUTX potently stabilizes and enhances the aberrant FSHD gene expression signature [kong_engineered_2024.```

In [7]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y.1_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.1_(Homo_sapiens),variant,Homo sapiens,9606,172,H3.Y.1_(Primates),172,H3.Y.1_(Homo_sapiens) is a human histone H3 va...,null,null,...,Knockdown of both genes encoding H3.Y (H3Y1 an...,Although the functions of H3.Y.1 remain poorly...,The H3.Y.1 protein consists of 135 amino acids...,Endogenous H3.Y.1 is predominantly localized o...,H3.Y.1 is deposited into chromatin via the HIR...,The crystal structure of the H3.Y.1 nucleosome...,null,null,null,None


In [9]:
knock_out_desc = "Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024]."
function_desc = "Although the functions of H3.Y.1 remain poorly understood, it is known to promote cell growth and regulate the expression of genes involved in cell cycle control and mitosis [wiedemann_identification_2010]. The presence of H3.Y.1 (along with H3.Y.2) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. Furthermore, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also detected in vivo within the nuclei of human 8-cell embryos at the peak of ZGA [taubenschmid-stowers_8c-like_2022]. During blastomere division, H3.Y.1 and H3.Y.2 show strong association with condensed chromosomes at prophase and metaphase stages [taubenschmid-stowers_8c-like_2022]. Taubenschmid-Stowers et al. suggest that H3.Y.1 and H3.Y.2 may be necessary for large-scale genome activation during early human embryogenesis [taubenschmid-stowers_8c-like_2022]."
disease_desc = "H3.Y.1 and H3.Y.2 are critical epigenetic amplifiers in facioscapulohumeral dystrophy (FSHD) pathogenesis [kong_engineered_2024]. They incorporate into chromatin to activate late targets like LEUTX, which in turn stimulates H3.Y.1 and H3.Y.2 expression, forming a pathogenic positive feedback loop. This coherent feedforward network with DUX4 and LEUTX potently stabilizes and enhances the aberrant FSHD gene expression signature [kong_engineered_2024]."
query = f"UPDATE histone_description SET knock_out='{knock_out_desc}', function='{function_desc}', disease='{disease_desc}' WHERE id=172"
print(query)
cursor.execute(query)

UPDATE histone_description SET knock_out='Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024].', function='Although the functions of H3.Y.1 remain poorly understood, it is known to promote cell growth and regulate the expression of genes involved in cell cycle control and mitosis [wiedemann_identification_2010]. The presence of H3.Y.1 (along with H3.Y.2) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. Furthermore, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, 

In [11]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y.1_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.1_(Homo_sapiens),variant,Homo sapiens,9606,172,H3.Y.1_(Primates),172,H3.Y.1_(Homo_sapiens) is a human histone H3 va...,null,null,...,Knockdown of both genes encoding H3.Y (H3Y1 an...,Although the functions of H3.Y.1 remain poorly...,The H3.Y.1 protein consists of 135 amino acids...,Endogenous H3.Y.1 is predominantly localized o...,H3.Y.1 is deposited into chromatin via the HIR...,The crystal structure of the H3.Y.1 nucleosome...,null,H3.Y.1 and H3.Y.2 are critical epigenetic ampl...,null,None


In [12]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Update description to H3.Y.2_(Homo_sapiens)</span>

### <span style="color:black">Update knock_out</span>

```Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024].```

### <span style="color:black">Update function</span>

```The functions of H3.Y.2 remain poorly understood. However, the presence of H3.Y.2 (along with H3.Y.1) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. In addition, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also detected in vivo within the nuclei of human 8-cell embryos at the peak of ZGA [taubenschmid-stowers_8c-like_2022]. During blastomere division, H3.Y.1 and H3.Y.2 show strong association with condensed chromosomes at prophase and metaphase stages [taubenschmid-stowers_8c-like_2022]. Taubenschmid-Stowers et al. suggest that H3.Y.1 and H3.Y.2 may be necessary for large-scale genome activation during early human embryogenesis [taubenschmid-stowers_8c-like_2022].```

### <span style="color:black">Add disease</span>

```H3.Y.1 and H3.Y.2 are critical epigenetic amplifiers in facioscapulohumeral dystrophy (FSHD) pathogenesis [kong_engineered_2024]. They incorporate into chromatin to activate late targets like LEUTX, which in turn stimulates H3.Y.1 and H3.Y.2 expression, forming a pathogenic positive feedback loop. This coherent feedforward network with DUX4 and LEUTX potently stabilizes and enhances the aberrant FSHD gene expression signature [kong_engineered_2024.```

In [13]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y.2_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.2_(Homo_sapiens),variant,Homo sapiens,9606,173,H3.Y.2_(Primates),173,H3.Y.2_(Homo_sapiens) is a human histone H3 va...,null,null,...,Knockdown of both genes encoding H3.Y (H3Y1 an...,The functions of H3.Y.2 remain poorly understo...,The H3.Y.2 protein consists of 146 amino acids...,H3.Y.1 and H3.Y.2 are incorporated into highly...,H3.Y.2 is deposited into chromatin via the HIR...,The crystal structure of the H3.Y.1 nucleosome...,null,null,null,None


In [15]:
knock_out_desc = "Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024]."
function_desc = "The functions of H3.Y.2 remain poorly understood. However, the presence of H3.Y.2 (along with H3.Y.1) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. In addition, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also detected in vivo within the nuclei of human 8-cell embryos at the peak of ZGA [taubenschmid-stowers_8c-like_2022]. During blastomere division, H3.Y.1 and H3.Y.2 show strong association with condensed chromosomes at prophase and metaphase stages [taubenschmid-stowers_8c-like_2022]. Taubenschmid-Stowers et al. suggest that H3.Y.1 and H3.Y.2 may be necessary for large-scale genome activation during early human embryogenesis [taubenschmid-stowers_8c-like_2022]."
disease_desc = "H3.Y.1 and H3.Y.2 are critical epigenetic amplifiers in facioscapulohumeral dystrophy (FSHD) pathogenesis [kong_engineered_2024]. They incorporate into chromatin to activate late targets like LEUTX, which in turn stimulates H3.Y.1 and H3.Y.2 expression, forming a pathogenic positive feedback loop. This coherent feedforward network with DUX4 and LEUTX potently stabilizes and enhances the aberrant FSHD gene expression signature [kong_engineered_2024]."
query = f"UPDATE histone_description SET knock_out='{knock_out_desc}', function='{function_desc}', disease='{disease_desc}' WHERE id=173"
print(query)
cursor.execute(query)

UPDATE histone_description SET knock_out='Knockdown of both genes encoding H3.Y (H3Y1 and H3Y2) using siRNA suppresses the super-induction of DUX4 target genes (including LEUTX) upon reactivation and reduces the persistence of their expression, but does not affect constitutively expressed genes [resnick_dux4-induced_2019, kong_engineered_2024].', function='The functions of H3.Y.2 remain poorly understood. However, the presence of H3.Y.2 (along with H3.Y.1) in hippocampal neurons suggests a potential role in primate-specific brain functions [wiedemann_identification_2010]. In addition, H3.Y.1 and H3.Y.2 are induced by the transcription factor DUX4, participant of zygotic genome activation (ZGA), to facilitate the persistence and reactivation of DUX4 target genes following its transient expression [kong_engineered_2024, talbert_histone_2021, resnick_dux4-induced_2019]. Interestinglly, H3.Y.1 and H3.Y.2 have been identified as a specific marker of 8-cell-like cells (8CLCs) and are also de

In [16]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y.2_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.2_(Homo_sapiens),variant,Homo sapiens,9606,173,H3.Y.2_(Primates),173,H3.Y.2_(Homo_sapiens) is a human histone H3 va...,null,null,...,Knockdown of both genes encoding H3.Y (H3Y1 an...,The functions of H3.Y.2 remain poorly understo...,The H3.Y.2 protein consists of 146 amino acids...,H3.Y.1 and H3.Y.2 are incorporated into highly...,H3.Y.2 is deposited into chromatin via the HIR...,The crystal structure of the H3.Y.1 nucleosome...,null,H3.Y.1 and H3.Y.2 are critical epigenetic ampl...,null,None


In [17]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add description to H3.4_(Homo_sapiens)</span>

### <span style="color:black">Add summary</span>

```H3.4_(Homo_sapiens) is a human H3 histone variant highly expressed in testis and at lower level in other tissues.```

### <span style="color:black">Add gene</span>

```H3.4 is encoded by H3-4 gene (formerly HIST3H3, also an alias name H3C16 has been suggested (Seal et al.)), located on chromosome 1 adjacent to genes H2AC25 and H2BC26 in a small histone gene cluster [kycia_tudor_2014]. This gene is intronless, its mRNA has a stem-loop structure.```

### <span style="color:black">Add deposition</span>

```Tachiwana et al. have found that H3.4/H4 is not efficiently incorporated into the nucleosome by human Nap1 chaperone, due to its defective H3.4/H4 deposition on DNA. In contrast, human Nap2 chaperone, a paralog of Nap1, promoted nucleosome assembly with H3.4/H4. Mutational analyses revealed that the A111V mutation in H3.4 is essential for this difference [tachiwana_nucleosome_2008].```

### <span style="color:black">Add structure</span>

```PDB structure of human H3.4 containing nucleosome is available with id 3A6N. Structural studies have revealed that H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].```

### <span style="color:black">Add sequence</span>

```In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V) [ueda_testis-specific_2017].```

### <span style="color:black">Add disease</span>

```The single‑nucleotide polymorphism c190C>T (Arg64Cys) in the human testis‑specific histone variant, H3.4, was studied in Japanese patients for its association with Sertoli cell‑only syndrome, which causes infertility [miyamoto_association_2018].```

In [19]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.4_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.4_(Homo_sapiens),variant,Homo sapiens,9606,107,H3.4_(Mammalia),107,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [20]:
df.to_dict(orient="records")

[{'id': 107,
  'level': 'variant',
  'taxonomic_span': 'Homo sapiens',
  'taxonomic_span_id': '9606',
  'description': 107,
  'parent': 'H3.4_(Mammalia)',
  'summary': 'null',
  'taxonomy': 'null',
  'genes': 'null',
  'evolution': 'null',
  'expression': 'null',
  'knock_out': 'null',
  'function': 'null',
  'sequence': 'null',
  'localization': 'null',
  'deposition': 'null',
  'structure': 'null',
  'interactions': 'null',
  'disease': 'null',
  'caveats': 'null',
  'relations': None}]

In [22]:
desc_dict = {
    "summary": "H3.4_(Homo_sapiens) is a human H3 histone variant highly expressed in testis and at lower level in other tissues.",
    "sequence": "In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V) [ueda_testis-specific_2017].",
    "deposition": "Tachiwana et al. have found that H3.4/H4 is not efficiently incorporated into the nucleosome by human Nap1 chaperone, due to its defective H3.4/H4 deposition on DNA. In contrast, human Nap2 chaperone, a paralog of Nap1, promoted nucleosome assembly with H3.4/H4. Mutational analyses revealed that the A111V mutation in H3.4 is essential for this difference [tachiwana_nucleosome_2008].",
    "structure": "PDB structure of human H3.4 containing nucleosome is available with id 3A6N. Structural studies have revealed that H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].",
    "disease": "The single‑nucleotide polymorphism c190C>T (Arg64Cys) in the human testis‑specific histone variant, H3.4, was studied in Japanese patients for its association with Sertoli cell‑only syndrome, which causes infertility [miyamoto_association_2018].",
}
desk_str = ", ".join([f'{k}="{v}"' for k, v in desc_dict.items()])
query = f"UPDATE histone_description SET {desk_str} " "WHERE id = 107"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary="H3.4_(Homo_sapiens) is a human H3 histone variant highly expressed in testis and at lower level in other tissues.", sequence="In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V) [ueda_testis-specific_2017].", deposition="Tachiwana et al. have found that H3.4/H4 is not efficiently incorporated into the nucleosome by human Nap1 chaperone, due to its defective H3.4/H4 deposition on DNA. In contrast, human Nap2 chaperone, a paralog of Nap1, promoted nucleosome assembly with H3.4/H4. Mutational analyses revealed that the A111V mutation in H3.4 is essential for this difference [tachiwana_nucleosome_2008].", structure="PDB structure of human H3.4 containing nucleosome is available with id 3A6N. Structural studies have revealed that H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].", disease="The single‑nucleotide polymorphism c190C>T (Arg64Cys) in the 

In [23]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.4_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.4_(Homo_sapiens),variant,Homo sapiens,9606,107,H3.4_(Mammalia),107,H3.4_(Homo_sapiens) is a human H3 histone vari...,null,null,...,null,null,In human H3.4 has only 4 amino acid difference...,null,Tachiwana et al. have found that H3.4/H4 is no...,PDB structure of human H3.4 containing nucleos...,null,The single‑nucleotide polymorphism c190C>T (Ar...,null,None


In [24]:
df.to_dict(orient="records")

[{'id': 107,
  'level': 'variant',
  'taxonomic_span': 'Homo sapiens',
  'taxonomic_span_id': '9606',
  'description': 107,
  'parent': 'H3.4_(Mammalia)',
  'summary': 'H3.4_(Homo_sapiens) is a human H3 histone variant highly expressed in testis and at lower level in other tissues.',
  'taxonomy': 'null',
  'genes': 'null',
  'evolution': 'null',
  'expression': 'null',
  'knock_out': 'null',
  'function': 'null',
  'sequence': 'In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V) [ueda_testis-specific_2017].',
  'localization': 'null',
  'deposition': 'Tachiwana et al. have found that H3.4/H4 is not efficiently incorporated into the nucleosome by human Nap1 chaperone, due to its defective H3.4/H4 deposition on DNA. In contrast, human Nap2 chaperone, a paralog of Nap1, promoted nucleosome assembly with H3.4/H4. Mutational analyses revealed that the A111V mutation in H3.4 is essential for this difference [tachiwana_nucleosome_2008].',
  'structure': 'PDB st

In [25]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Update description to H3.4_(Mammalia)</span>

### <span style="color:black">Update summary</span>

```H3.4_(Mammalia) is a mammal-specific H3 histone variant highly expressed in testis and at lower level in other tissues [kycia_tudor_2014]. This variant is also reffered to as H3.1t, H3T, or TS H3.4. The H3.4 variant is very similar to canonical H3 histones both in sequence, gene location and transcriptional regulation. In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V), its gene is located within histone gene cluster, is intronless, its mRNA has a stem-loop structure. H3.4 expression is replication-dependent [kycia_tudor_2014, ueda_testis-specific_2017].```

### <span style="color:black">Update knock_out</span>

```Knockout mice for H3.4 were first generated in 2017; both male and female H3t null mice were viable and healthy, but the male mice were sterile. H3.4 deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024, ueda_testis-specific_2017].```

### <span style="color:black">Update function</span>

```H3.4 is mainly responsible for facilitating the transition from spermatogonia stem cells to active spermatogenesis. Mouse H3.4 was shown to be essential for very early stages of spermatogenesis, its deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024]. When differentiating spermatogonia emerge in normal spermatogenesis, H3.4 appears and replaces the canonical H3 proteins [ueda_testis-specific_2017].```

### <span style="color:black">Update expression</span>

```Expression of H3.4 is strictly restricted to testicular tissue. In mice, it is detected in spermatogonia, meiotic cells, and haploid round spermatids. However, its expression level decreases significantly in early elongating spermatids, indicating stage-specific regulation during spermatogenesis [kawaguchi_comprehensive_2024].```

### <span style="color:black">Update localization</span>

```In mouse H3.4 is not incorporated into the XY body, which forms during pachytene of meiotic prophase I, suggesting a potential role for H3.4 in organizing active chromatin regions [kawaguchi_comprehensive_2024].```

### <span style="color:black">Update interactions</span>

```It was demostrated that A23V mutation favors the interaction of trymethyllysine-binding Tudor domains of PHF1 and PHF2 proteins (critical components of the PCR2 complex) with H3K27me3 marks. PHF1 co-localizes with H3.4 in testis and its Tudor domain preferentially binds to H3.4K27me3 over canonical H3K27me3 in vitro, implicating that H3.4K27me3 might be a physiological ligand of PHF1/19. [dong_structural_2020, kycia_tudor_2014].```

### <span style="color:black">Add structure</span>

```Structural studies have revealed that human H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].```

### <span style="color:black">Update disease to NULL</span>

In [26]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.4_(Mammalia)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.4_(Mammalia),variant_group,Mammalia,40674,51,H3,51,H3.4_(Mammalia) is a mammal-specific H3 histo...,Mammals,In human is encoded by H3-4 gene (formerly HIS...,...,Knockout mice for H3.4 were first generated in...,null,null,null,null,PDB structure of human H3.4 containing nucleos...,null,The single‑nucleotide polymorphism c190C>T (Ar...,null,None


In [27]:
df.to_dict(orient="records")

[{'id': 51,
  'level': 'variant_group',
  'taxonomic_span': 'Mammalia',
  'taxonomic_span_id': '40674',
  'description': 51,
  'parent': 'H3',
  'summary': 'H3.4_(Mammalia)  is a mammal-specific H3 histone variant highly expressed in testis and at lower level in other tissues [kycia_tudor_2014]. This variant is also reffered to as H3.1t, H3T, or TS H3.4. The H3.4 variant is very similar to canonical H3 histones both in sequence, gene location and transcriptional regulation. In human H3.4 has only 4 amino acid difference from cH3s (A23V,V71M, A98S, A111V), its gene is located within histone gene cluster, is intronless, its mRNA has a stem-loop structure. H3.4 expression is  replication-dependent [ueda_testis-specific_2017]. Mouse H3.4 was shown to be essential for very early stages of spermatogenesis, its deficiency leads to azoospermia because of the loss of haploid germ cells. When differentiating spermatogonia emerge in normal spermatogenesis, H3.4 appears and replaces the canonical 

In [29]:
desc_dict = {
    "summary": "H3.4_(Mammalia) is a mammal-specific H3 histone variant highly expressed in testis and at lower level in other tissues [kycia_tudor_2014]. This variant is also reffered to as H3.1t, H3T, or TS H3.4. The H3.4 variant is very similar to canonical H3 histones both in sequence, gene location and transcriptional regulation. In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V), its gene is located within histone gene cluster, is intronless, its mRNA has a stem-loop structure. H3.4 expression is replication-dependent [kycia_tudor_2014, ueda_testis-specific_2017].",
    "knock_out": "Knockout mice for H3.4 were first generated in 2017; both male and female H3t null mice were viable and healthy, but the male mice were sterile. H3.4 deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024, ueda_testis-specific_2017].",
    "function": "H3.4 is mainly responsible for facilitating the transition from spermatogonia stem cells to active spermatogenesis. Mouse H3.4 was shown to be essential for very early stages of spermatogenesis, its deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024]. When differentiating spermatogonia emerge in normal spermatogenesis, H3.4 appears and replaces the canonical H3 proteins [ueda_testis-specific_2017].",
    "expression": "Expression of H3.4 is strictly restricted to testicular tissue. In mice, it is detected in spermatogonia, meiotic cells, and haploid round spermatids. However, its expression level decreases significantly in early elongating spermatids, indicating stage-specific regulation during spermatogenesis [kawaguchi_comprehensive_2024].",
    "localization": "In mouse H3.4 is not incorporated into the XY body, which forms during pachytene of meiotic prophase I, suggesting a potential role for H3.4 in organizing active chromatin regions [kawaguchi_comprehensive_2024].",
    "interactions": "It was demostrated that A23V mutation favors the interaction of trymethyllysine-binding Tudor domains of PHF1 and PHF2 proteins (critical components of the PCR2 complex) with H3K27me3 marks. PHF1 co-localizes with H3.4 in testis and its Tudor domain preferentially binds to H3.4K27me3 over canonical H3K27me3 in vitro, implicating that H3.4K27me3 might be a physiological ligand of PHF1/19. [dong_structural_2020, kycia_tudor_2014].",
    "structure": "Structural studies have revealed that human H3.4 containing nucleosome are less stable, particularly due to V71M and A111V substitutions [tachiwana_structural_2010].",
}
desk_str = ", ".join([f'{k}="{v}"' for k, v in desc_dict.items()])
query = f"UPDATE histone_description SET {desk_str} " "WHERE id = 51"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary="H3.4_(Mammalia) is a mammal-specific H3 histone variant highly expressed in testis and at lower level in other tissues [kycia_tudor_2014]. This variant is also reffered to as H3.1t, H3T, or TS H3.4. The H3.4 variant is very similar to canonical H3 histones both in sequence, gene location and transcriptional regulation. In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V), its gene is located within histone gene cluster, is intronless, its mRNA has a stem-loop structure. H3.4 expression is replication-dependent [kycia_tudor_2014, ueda_testis-specific_2017].", knock_out="Knockout mice for H3.4 were first generated in 2017; both male and female H3t null mice were viable and healthy, but the male mice were sterile. H3.4 deficiency leads to azoospermia because of the loss of haploid germ cells [kawaguchi_comprehensive_2024, ueda_testis-specific_2017].", function="H3.4 is mainly responsible for facilitating the transition 

In [30]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.4_(Mammalia)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.4_(Mammalia),variant_group,Mammalia,40674,51,H3,51,H3.4_(Mammalia) is a mammal-specific H3 histon...,Mammals,In human is encoded by H3-4 gene (formerly HIS...,...,Knockout mice for H3.4 were first generated in...,H3.4 is mainly responsible for facilitating th...,null,In mouse H3.4 is not incorporated into the XY ...,null,Structural studies have revealed that human H3...,It was demostrated that A23V mutation favors t...,The single‑nucleotide polymorphism c190C>T (Ar...,null,None


In [31]:
df.to_dict(orient="records")

[{'id': 51,
  'level': 'variant_group',
  'taxonomic_span': 'Mammalia',
  'taxonomic_span_id': '40674',
  'description': 51,
  'parent': 'H3',
  'summary': 'H3.4_(Mammalia) is a mammal-specific H3 histone variant highly expressed in testis and at lower level in other tissues [kycia_tudor_2014]. This variant is also reffered to as H3.1t, H3T, or TS H3.4. The H3.4 variant is very similar to canonical H3 histones both in sequence, gene location and transcriptional regulation. In human H3.4 has only 4 amino acid difference from cH3s (A23V, V71M, A98S, A111V), its gene is located within histone gene cluster, is intronless, its mRNA has a stem-loop structure. H3.4 expression is replication-dependent [kycia_tudor_2014, ueda_testis-specific_2017].',
  'taxonomy': 'Mammals',
  'genes': 'In human is encoded by H3-4 gene (formerly HIST3H3, also an alias name H3C16 has been suggested (Seal et al.)), in mouse by H3f4 gene. The gene encodes mRNA with a stem loop structure. In human the gene is locat

In [34]:
query = f"UPDATE histone_description SET disease=null " "WHERE id = 51"
print(query)
cursor.execute(query)

UPDATE histone_description SET disease=null WHERE id = 51


In [36]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.4_(Mammalia)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.4_(Mammalia),variant_group,Mammalia,40674,51,H3,51,H3.4_(Mammalia) is a mammal-specific H3 histon...,Mammals,In human is encoded by H3-4 gene (formerly HIS...,...,Knockout mice for H3.4 were first generated in...,H3.4 is mainly responsible for facilitating th...,null,In mouse H3.4 is not incorporated into the XY ...,null,Structural studies have revealed that human H3...,It was demostrated that A23V mutation favors t...,None,null,None


In [35]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Update description to H3.5_(Hominidae)</span>

### <span style="color:black">Update summary</span>

```H3.5 is a hominid-specific histone H3 variant, also known as H3.3C, expressed in the seminiferous tubules of the human testis and plays a role in regulating spermatogenesis. H3.5 likely evolved from H3F3B gene duplication and is associated with euchromatin and actively transcribed genes [schenk_h35_2011].```

### <span style="color:black">Update sequence</span>

```H3.3 is the most similar variant to H3.5 (approximately 96% identity). H3.5 has two copies of the ARKST motif and 5 amino acids differences comparing to H3.3 [schenk_h35_2011, ederveen_human_2011].```

In [37]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.5_(Hominidae)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.5_(Hominidae),variant_group,null,null,206,H3,206,H3.5 is a hominid-specific histone H3 variant ...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [38]:
df.to_dict(orient="records")

[{'id': 206,
  'level': 'variant_group',
  'taxonomic_span': 'null',
  'taxonomic_span_id': 'null',
  'description': 206,
  'parent': 'H3',
  'summary': "H3.5 is a hominid-specific histone H3 variant expressed in the seminiferous tubules of human testes, encoded on chromosome 12p11.21 region of human chromosome 12. H3.3 is the most similar variant to H3.5 (approximately 96% identity). H3.5 has two copies of the ARKST motif and 5 amino acids differences comparing to H3.3 [schenk_h35_2011, ederveen_human_2011]. H3.5 likely evolved from H3F3B gene duplication and is associated with euchromatin and actively transcribed genes [schenk_h35_2011]. Nucleosomes containing histone variant H3.5 are less stable than H3.1 and H3.3 nucleosomes, but more stable than H3.4 nucleosome [urahama_histone_2016]. Notable, amino acid L in position 103 of H3.5 (amino acid F at the corresponding position in H3.3) reduces the number of hydrophobic bonds with histone H4 leading to instability of nucleosome [uraham

In [39]:
desc_dict = {
    "summary": "H3.5 is a hominid-specific histone H3 variant, also known as H3.3C, expressed in the seminiferous tubules of the human testis and plays a role in regulating spermatogenesis. H3.5 likely evolved from H3F3B gene duplication and is associated with euchromatin and actively transcribed genes [schenk_h35_2011].",
    "sequence": "H3.3 is the most similar variant to H3.5 (approximately 96% identity). H3.5 has two copies of the ARKST motif and 5 amino acids differences comparing to H3.3 [schenk_h35_2011, ederveen_human_2011].",
}
desk_str = ", ".join([f'{k}="{v}"' for k, v in desc_dict.items()])
query = f"UPDATE histone_description SET {desk_str} " "WHERE id = 206"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary="H3.5 is a hominid-specific histone H3 variant, also known as H3.3C, expressed in the seminiferous tubules of the human testis and plays a role in regulating spermatogenesis. H3.5 likely evolved from H3F3B gene duplication and is associated with euchromatin and actively transcribed genes [schenk_h35_2011].", sequence="H3.3 is the most similar variant to H3.5 (approximately 96% identity). H3.5 has two copies of the ARKST motif and 5 amino acids differences comparing to H3.3 [schenk_h35_2011, ederveen_human_2011]." WHERE id = 206


In [40]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.5_(Hominidae)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.5_(Hominidae),variant_group,null,null,206,H3,206,"H3.5 is a hominid-specific histone H3 variant,...",None,None,...,None,None,H3.3 is the most similar variant to H3.5 (appr...,None,None,None,None,None,None,None


In [41]:
df.to_dict(orient="records")

[{'id': 206,
  'level': 'variant_group',
  'taxonomic_span': 'null',
  'taxonomic_span_id': 'null',
  'description': 206,
  'parent': 'H3',
  'summary': 'H3.5 is a hominid-specific histone H3 variant, also known as H3.3C, expressed in the seminiferous tubules of the human testis and plays a role in regulating spermatogenesis. H3.5 likely evolved from H3F3B gene duplication and is associated with euchromatin and actively transcribed genes [schenk_h35_2011].',
  'taxonomy': None,
  'genes': None,
  'evolution': None,
  'expression': None,
  'knock_out': None,
  'function': None,
  'sequence': 'H3.3 is the most similar variant to H3.5 (approximately 96% identity). H3.5 has two copies of the ARKST motif and 5 amino acids differences comparing to H3.3 [schenk_h35_2011, ederveen_human_2011].',
  'localization': None,
  'deposition': None,
  'structure': None,
  'interactions': None,
  'disease': None,
  'caveats': None,
  'relations': None}]

In [42]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add description to H3.5_(Homo_sapiens)</span>

### <span style="color:black">Add summary</span>

```H3.5_(Homo_sapiens) is a H3 histone variant expressed in the seminiferous tubules of human testes.```

### <span style="color:black">Add function</span>

```H3.5 plays a significant role in testicular cell differentiation, with potential implications in the pathogenesis of testicular diseases.```

### <span style="color:black">Add expression</span>

```The highest expression of H3.5 is found in spermatocytes, while lower levels are present in early and late spermatids, and absent in mature spermatozoa [weil_histone_2024, ding_primate-specific_2021]. However, low-abundance H3.5 transcripts were also identified in leukocytes and hepatocytes [schenk_h35_2011].```

### <span style="color:black">Add gene</span>

```H3.5 is encoded by H3-5 gene on chromosome 12. Methylation of the CpG island in the promoter region of the gene plays a key role in its tissue-specific expression regulation [weil_histone_2024].```

### <span style="color:black">Update sequence</span>

```H3.5 differs from H3.3 by five amino acid substitutions: Thr29, Cys33, Asn78, Val88, and Leu103 (which correspond to Ala29, Gly33, Lys79, Ile89, and Phe104 in H3.3, respectively) [urahama_histone_2016]. Furthermore, H3.5 lacks the lysine residue at position 37, which is present in the major H3 variants.```

### <span style="color:black">Update localization</span>

```H3.5 localizes primarely at exons, 5'UTR, 3'UTR, promoters and introns. Although it enreached around transcription start sites (TSSs), H3.5 distribution is independent of gene expression levels [urahama_histone_2016].```

### <span style="color:black">Add deposition</span>

```H3.5 physically interacts with histone chaperones Asf1a, Asf1b, HIRA, CAF-1 p150, and DAXX, suggesting its involvement in chromatin remodeling and gene regulation processes [weil_histone_2024]. An unpublished experiment by Ding et al. demonstrated that the chaperone NAP1L1 exhibits a higher binding affinity for the H3.5/H4 tetramer compared to the H3.3/H4 tetramer, attributable to amino acid alterations within the N-terminal region of H3.5 [ding_primate-specific_2021].```

### <span style="color:black">Update structure</span>

```Nucleosomes containing histone variant H3.5 are less stable than H3.1 and H3.3 nucleosomes, but more stable than H3.4 nucleosome [urahama_histone_2016]. Notable, amino acid L in position 103 of H3.5 (amino acid F at the corresponding position in H3.3) reduces the number of hydrophobic bonds with histone H4 leading to instability of nucleosome [urahama_histone_2016].```

### <span style="color:black">Update disease</span>

```Clinical data indicate that H3.5 mRNA levels are significantly lower in patients with nonobstructive azoospermia compared to those with obstructive azoospermia or normal individuals, suggesting H3.5's role in normal spermatogenesis and its regulation by gonadotropins [ding_primate-specific_2021]. Elevated H3.5 expression is observed in seminomas, and H3-5 gene copy number gain is associated with its increased expression in tumor tissues [weil_histone_2024].```

In [43]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.5_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.5_(Homo_sapiens),variant,Homo sapiens,9606,108,H3.5_(Hominidae),108,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [44]:
df.to_dict(orient="records")

[{'id': 108,
  'level': 'variant',
  'taxonomic_span': 'Homo sapiens',
  'taxonomic_span_id': '9606',
  'description': 108,
  'parent': 'H3.5_(Hominidae)',
  'summary': 'null',
  'taxonomy': 'null',
  'genes': 'null',
  'evolution': 'null',
  'expression': 'null',
  'knock_out': 'null',
  'function': 'null',
  'sequence': 'null',
  'localization': 'null',
  'deposition': 'null',
  'structure': 'null',
  'interactions': 'null',
  'disease': 'null',
  'caveats': 'null',
  'relations': None}]

In [46]:
desc_dict = {
    "summary": "H3.5_(Homo_sapiens) is a H3 histone variant expressed in the seminiferous tubules of human testes.",
    "genes": "H3.5 is encoded by H3-5 gene on chromosome 12. Methylation of the CpG island in the promoter region of the gene plays a key role in its tissue-specific expression regulation [weil_histone_2024].",
    "expression": "The highest expression of H3.5 is found in spermatocytes, while lower levels are present in early and late spermatids, and absent in mature spermatozoa [weil_histone_2024, ding_primate-specific_2021]. However, low-abundance H3.5 transcripts were also identified in leukocytes and hepatocytes [schenk_h35_2011].",
    "function": "H3.5 plays a significant role in testicular cell differentiation, with potential implications in the pathogenesis of testicular diseases.",
    "sequence": "H3.5 differs from H3.3 by five amino acid substitutions: Thr29, Cys33, Asn78, Val88, and Leu103 (which correspond to Ala29, Gly33, Lys79, Ile89, and Phe104 in H3.3, respectively) [urahama_histone_2016]. Furthermore, H3.5 lacks the lysine residue at position 37, which is present in the major H3 variants.",
    "localization": "H3.5 localizes primarely at exons, 5'UTR, 3'UTR, promoters and introns. Although it enreached around transcription start sites (TSSs), H3.5 distribution is independent of gene expression levels [urahama_histone_2016].",
    "deposition": "H3.5 physically interacts with histone chaperones Asf1a, Asf1b, HIRA, CAF-1 p150, and DAXX, suggesting its involvement in chromatin remodeling and gene regulation processes [weil_histone_2024]. An unpublished experiment by Ding et al. demonstrated that the chaperone NAP1L1 exhibits a higher binding affinity for the H3.5/H4 tetramer compared to the H3.3/H4 tetramer, attributable to amino acid alterations within the N-terminal region of H3.5 [ding_primate-specific_2021].",
    "structure": "Nucleosomes containing histone variant H3.5 are less stable than H3.1 and H3.3 nucleosomes, but more stable than H3.4 nucleosome [urahama_histone_2016]. Notable, amino acid L in position 103 of H3.5 (amino acid F at the corresponding position in H3.3) reduces the number of hydrophobic bonds with histone H4 leading to instability of nucleosome [urahama_histone_2016].",
    "disease": "Clinical data indicate that H3.5 mRNA levels are significantly lower in patients with nonobstructive azoospermia compared to those with obstructive azoospermia or normal individuals, suggesting H3.5's role in normal spermatogenesis and its regulation by gonadotropins [ding_primate-specific_2021]. Elevated H3.5 expression is observed in seminomas, and H3-5 gene copy number gain is associated with its increased expression in tumor tissues [weil_histone_2024].",
}
desk_str = ", ".join([f'{k}="{v}"' for k, v in desc_dict.items()])
query = f"UPDATE histone_description SET {desk_str} " "WHERE id = 108"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary="H3.5_(Homo_sapiens) is a H3 histone variant expressed in the seminiferous tubules of human testes.", genes="H3.5 is encoded by H3-5 gene on chromosome 12. Methylation of the CpG island in the promoter region of the gene plays a key role in its tissue-specific expression regulation [weil_histone_2024].", expression="The highest expression of H3.5 is found in spermatocytes, while lower levels are present in early and late spermatids, and absent in mature spermatozoa [weil_histone_2024, ding_primate-specific_2021]. However, low-abundance H3.5 transcripts were also identified in leukocytes and hepatocytes [schenk_h35_2011].", function="H3.5 plays a significant role in testicular cell differentiation, with potential implications in the pathogenesis of testicular diseases.", sequence="H3.5 differs from H3.3 by five amino acid substitutions: Thr29, Cys33, Asn78, Val88, and Leu103 (which correspond to Ala29, Gly33, Lys79, Ile89, and Phe104 in H3.3, respe

In [47]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.5_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.5_(Homo_sapiens),variant,Homo sapiens,9606,108,H3.5_(Hominidae),108,H3.5_(Homo_sapiens) is a H3 histone variant ex...,null,H3.5 is encoded by H3-5 gene on chromosome 12....,...,null,H3.5 plays a significant role in testicular ce...,H3.5 differs from H3.3 by five amino acid subs...,"H3.5 localizes primarely at exons, 5'UTR, 3'UT...",H3.5 physically interacts with histone chapero...,Nucleosomes containing histone variant H3.5 ar...,null,Clinical data indicate that H3.5 mRNA levels a...,null,None


In [48]:
df.to_dict(orient="records")

[{'id': 108,
  'level': 'variant',
  'taxonomic_span': 'Homo sapiens',
  'taxonomic_span_id': '9606',
  'description': 108,
  'parent': 'H3.5_(Hominidae)',
  'summary': 'H3.5_(Homo_sapiens) is a H3 histone variant expressed in the seminiferous tubules of human testes.',
  'taxonomy': 'null',
  'genes': 'H3.5 is encoded by H3-5 gene on chromosome 12. Methylation of the CpG island in the promoter region of the gene plays a key role in its tissue-specific expression regulation [weil_histone_2024].',
  'evolution': 'null',
  'expression': 'The highest expression of H3.5 is found in spermatocytes, while lower levels are present in early and late spermatids, and absent in mature spermatozoa [weil_histone_2024, ding_primate-specific_2021]. However, low-abundance H3.5 transcripts were also identified in leukocytes and hepatocytes [schenk_h35_2011].',
  'knock_out': 'null',
  'function': 'H3.5 plays a significant role in testicular cell differentiation, with potential implications in the pathog

In [49]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [50]:
cursor.close()
conn.close()
tunnel.stop()